In [0]:
import boto3
import os

In [0]:
df_customers = spark.read.format("delta").load("/Volumes/workspace/default/etl_path/silver_layer/customers")
df_employees = spark.read.format("delta").load("/Volumes/workspace/default/etl_path/silver_layer/employees")
df_sales = spark.read.format("delta").load("/Volumes/workspace/default/etl_path/silver_layer/sales")

In [0]:
df_customers.coalesce(1).write.mode("overwrite").parquet("/Volumes/workspace/default/etl_path/gold_layer/customers")
df_sales.coalesce(1).write.mode("overwrite").parquet("/Volumes/workspace/default/etl_path/gold_layer/sales")
df_employees.coalesce(1).write.mode("overwrite").parquet("/Volumes/workspace/default/etl_path/gold_layer/employees")

In [0]:
scope_name = 'aws-keys' # Your scope name
ACCESS_KEY = dbutils.secrets.get(scope=scope_name, key="AWS_ACCESS_KEY_ID") # Your AWS Access Key
SECRET_KEY = dbutils.secrets.get(scope=scope_name, key="AWS_SECRET_ACCESS_KEY") # Your AWS Secret Key
AWS_BUCKET_NAME = "csv-databricks-etl" # Your S3 Bucket Name

s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)

base_path = '/Volumes/workspace/default/etl_path/gold_layer/'

folders = ['customers', 'sales', 'employees']

for folder in folders:
    folder_path = os.path.join(base_path, folder)
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            local_file_path = os.path.join(root, file_name)
            relative_path = os.path.relpath(local_file_path, base_path)
            s3_key = f"gold_layer/{relative_path.replace(os.sep, '/')}"
            with open(local_file_path, 'rb') as data:
                s3.upload_fileobj(data, AWS_BUCKET_NAME, s3_key)
            print(f"Uploaded {file_name} to s3://{AWS_BUCKET_NAME}/{s3_key}")